In [5]:
import torch
from torch import nn
from resnet import BasicBlock

In [2]:
x = torch.rand((1,64,16,16))
print(x.shape)

torch.Size([1, 64, 16, 16])


In [27]:
def combine_function_1(x, output):
    x = nn.Sequential(
        nn.Conv2d(64,64,3,stride=2,padding=1)
    )(x)
    return x + output

def combine_function_2(x, output):
    x = nn.Sequential(
        nn.Conv2d(64,64,2,stride=2,padding=0)
    )(x)
    print(x.shape)
    return x + output

In [24]:
# Combination Function: Conv of x to reduce width*height by half to match y
def combine_function(x, output):
    x = nn.Sequential(
        nn.Conv2d(64,64,2,stride=2,padding=0)
    )(x)
    print(x.shape)
    return x + output

In [25]:
basic_block = BasicBlock(
    in_channels=64,
    out_channels=64,
    stride=2,
    combine_function=combine_function,
)

In [26]:
y = basic_block.forward(x)

torch.Size([1, 64, 8, 8])


In [ ]:
print(f"x shape: {x.shape}\ny shape: {y.shape}")

x shape: torch.Size([1, 64, 16, 16])
y shape: torch.Size([1, 64, 8, 8])
